In [1]:
import numpy as np

import sys
sys.path.append('../../src/phloemfinder/')

from feature_selection_using_ml import MetabolitePhenotypeFeatureSelection 

/Users/lissydenkers/miniconda3/envs/sklearn/lib/python3.9/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [10]:
micromass = MetabolitePhenotypeFeatureSelection(
    metabolome_csv="./filtered_features.csv",    
    phenotype_csv="./phenotype.csv",
    phenotype_sample_id='sample_id')

In [11]:
micromass.validate_input_metabolome_df()
micromass.validate_input_phenotype_df()

Metabolome data validated.
Phenotype data validated.


In [12]:
print(micromass.metabolome.shape)
print(micromass.phenotype.shape)

(599, 571)
(571, 1)


In [13]:
micromass.get_baseline_performance(
    train_size=0.7,
    random_state=123)

====== Training a basic Random Forest model =======
Average balanced_accuracy score on training data is: 97.200 % -/+ 2.00


====== Performance on test data of the basic Random Forest model =======
Average balanced_accuracy score on test data is: 99.100 %


In [15]:
micromass.search_best_model_with_tpot_and_compute_pc_importances(
    class_of_interest='positive',
    max_time_mins=30,
    max_eval_time_mins=6,
    random_state=123)

Version 0.12.0 of tpot is outdated. Version 0.12.2 was released Friday February 23, 2024.


                                                                                
57.03 minutes have elapsed. TPOT will close down.                               
TPOT closed during evaluation in one generation.
                                                                                
                                                                                
TPOT closed prematurely. Will use the current best pipeline.
                                                                                
Best pipeline: GradientBoostingClassifier(MaxAbsScaler(input_matrix), learning_rate=0.5, max_depth=3, max_features=0.35000000000000003, min_samples_leaf=4, min_samples_split=10, n_estimators=1000, subsample=1.0)
============ Performance of ML model on train data =============
Train balanced_accuracy score 100.000 %


============ Performance of ML model on test data =============
                   value
balanced_accuracy  0.942
precision          0.833
recall             0.972
f1

In [16]:
micromass.best_model

Pipeline(steps=[('maxabsscaler', MaxAbsScaler()),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(learning_rate=0.5,
                                            max_features=0.35000000000000003,
                                            min_samples_leaf=4,
                                            min_samples_split=10,
                                            n_estimators=1000,
                                            random_state=123))])

In [17]:
print(micromass.pc_importances)

       mean_var_imp  std_var_imp    perm0     perm1     perm2     perm3  \
pc                                                                        
PC0        0.463219     0.016346  0.46079  0.476562  0.487673  0.453698   
PC383      0.000000     0.000000  0.00000  0.000000  0.000000  0.000000   
PC377      0.000000     0.000000  0.00000  0.000000  0.000000  0.000000   
PC378      0.000000     0.000000  0.00000  0.000000  0.000000  0.000000   
PC379      0.000000     0.000000  0.00000  0.000000  0.000000  0.000000   
...             ...          ...      ...       ...       ...       ...   
PC184      0.000000     0.000000  0.00000  0.000000  0.000000  0.000000   
PC183      0.000000     0.000000  0.00000  0.000000  0.000000  0.000000   
PC182      0.000000     0.000000  0.00000  0.000000  0.000000  0.000000   
PC181      0.000000     0.000000  0.00000  0.000000  0.000000  0.000000   
PC570      0.000000     0.000000  0.00000  0.000000  0.000000  0.000000   

          perm4     perm

In [18]:
micromass.get_names_of_top_n_features_from_selected_pc(
    selected_pc=1,
    top_n=15)

Here are the metabolite names with the top 15 absolute loadings on PC1


,feature_name,loading
0,feature_347,0.518913
1,feature_1129,0.400986
2,feature_326,0.300760
3,feature_1080,0.298262
4,feature_1181,0.264013
5,feature_1261,0.260016
6,feature_1163,0.207912
7,feature_1002,0.153937
8,feature_579,0.130690
9,feature_128,0.129153


In [20]:
caddisfly_chemistry.get_names_of_top_n_features_from_selected_pc(
    selected_pc=21,
    top_n=15)

Here are the metabolite names with the top 15 absolute loadings on PC21


['random34',
 'random50',
 'random142',
 'random138',
 'hco',
 'random170',
 'random46',
 'random26',
 'random166',
 'random98',
 'random134',
 'random158',
 'random126',
 'random90',
 'random198']